In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd
import importlib
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated
import general_functions.databricks_connection as db
from general_functions.sanitize_accout_name import sanitize_account_name

In [3]:
url = return_api_url()
print(f"url = {url}")
accounts = return_account_ids()

# Modify Job Tags

In [69]:
importlib.reload(db)
db_connection = db.DatabricksClient()
jobs = db_connection.return_jobs()
print(len(jobs), jobs)

In [65]:
no_action_defined = []
for job in jobs:
    try:
        job_name = job["settings"]["name"]
    except KeyError:
        no_action_defined.append(job)
        continue
    found_job = False
    for account in accounts:
        account_name = account["name"]
        sanitzed_account_name = sanitize_account_name(account_name)
        print(f"Account: {account_name}, Sanitized: {sanitzed_account_name}")
        if sanitzed_account_name in job_name:
            job["settings"]["tags"]["account"] = account_name
            update_job_config = {
                "job_id": job["job_id"],
                "new_settings": {
                    "tags": job["settings"]["tags"]
                }
            }
            #print(f"Update job config {job_name}: {update_job_config}")
           #db_connection.update_job(update_job_config)
            found_job = True
            break
    if found_job == False:
        no_action_defined.append(job)


In [67]:
for job in no_action_defined:
    job_name = job["settings"]["name"]
    print(job_name)
    job_tag = False
    if "dev" in job_name:
        job["settings"]["tags"]["account"] = "Dev"
        job_tag = True
    elif "monitoring" in job_name or "write-datashifts-to-table" in job_name or "log_active_prefect_runs" in job_name or "log_daily_audiences" in job_name:
        job["settings"]["tags"]["account"] = "Overhead"
        job_tag = True
    if job_tag:
        update_job_config = {
                "job_id": job["job_id"],
                "new_settings": {
                    "tags": job["settings"]["tags"]
                }
            }
        print(f"Update job config {job_name}: {update_job_config}")
        db_connection.update_job(update_job_config)
    else:
        print(". No tagging")


# Modify Cluster Tags

In [83]:
importlib.reload(db)
databricks_connection = db.DatabricksClient()
clusters = databricks_connection.return_clusters()
len(clusters)

In [85]:
no_cluster_action_defined = []
for cluster in clusters:
    cluster_name = cluster["cluster_name"]
    cluster_id = cluster["cluster_id"]
    print(f"cluster_name: {cluster_name}, cluster_id: {cluster_id}")
    update_config = False
    for account in accounts:
        account_name = account["name"]
        sanitzed_account_name = sanitize_account_name(account_name)
        if sanitzed_account_name in cluster_name or account_name in cluster_name:
            cluster_info = databricks_connection.return_custer_info(cluster_id)
            cluster_info["custom_tags"]["account"] = account_name
            cluster_info.pop("disk_spec")
            print(f"Update cluster config {cluster_name}: {cluster_info}")
            databricks_connection.update_cluster(cluster_info)
            update_config = True
            break
    if "Notebook Cluster" in cluster_name:
            cluster_info = databricks_connection.return_custer_info(cluster_id)
            cluster_info["custom_tags"]["account"] = "Overhead"
            cluster_info.pop("disk_spec")
            print(f"Update Noteobook cluster config {cluster_name}: {cluster_info}")
            databricks_connection.update_cluster(cluster_info)
            update_config = True
    if update_config == False:
        no_cluster_action_defined.append(cluster)

In [89]:
for cluster in no_cluster_action_defined:
    cluster_name = cluster["cluster_name"]
    print(f"cluster_name: {cluster_name}")
    cluster_id = cluster["cluster_id"]
    cluster_info = databricks_connection.return_custer_info(cluster_id)
    if "Asam" in cluster_name:
        account_name = "Asambeauty"
    else:
        account_name = "Overhead"
    cluster_info["custom_tags"]["account"] = account_name
    cluster_info.pop("disk_spec")
    print(f"Update cluster config {cluster_name}: {cluster_info}")
    databricks_connection.update_cluster(cluster_info)